In [ ]:
!pip install -q bitsandbytes datasets accelerate loralib evaluate trl  bitsandbytes peft
!pip install -q git+https://github.com/huggingface/peft.git git+https://github.com/huggingface/transformers.git
!pip install --upgrade --no-cache-dir --no-deps unsloth


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 31.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 348.0/348.0 kB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 52.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 92.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 42.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# Fine-tuning con LoRA de Salamandra-2b instruct
A continuación haremos un fine-tuning de Salamandra 2B usando LoRA.

In [ ]:
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM
from datetime import datetime
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch
import torch.nn as nn
import bitsandbytes as bnb
import pandas as pd

from datasets import load_dataset
from peft import LoraConfig, get_peft_model

import os

os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

# 1. Cargamos los datos
Stanford Question Answering Dataset (SQuAD): conjunto de datos de comprensión lectora, que consta de preguntas planteadas por trabajadores colaborativos sobre un conjunto de artículos de Wikipedia, donde la respuesta a cada pregunta es un segmento de texto, o lapso, del pasaje de lectura correspondiente, o la pregunta podría no tener respuesta.
https://paperswithcode.com/dataset/squad

In [ ]:


qa_dataset = load_dataset("squad_v2")

README.md:   0%|          | 0.00/8.92k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/16.4M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.35M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/130319 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11873 [00:00<?, ? examples/s]

In [ ]:
print(qa_dataset)


DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 130319
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 11873
    })
})


In [ ]:
pd.DataFrame(qa_dataset["train"][:5])

,id,title,context,question,answers
0,56be85543aeaaa14008c9063,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce start becoming popular?,"{'text': ['in the late 1990s'], 'answer_start'..."
1,56be85543aeaaa14008c9065,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,What areas did Beyonce compete in when she was...,"{'text': ['singing and dancing'], 'answer_star..."
2,56be85543aeaaa14008c9066,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce leave Destiny's Child and bec...,"{'text': ['2003'], 'answer_start': [526]}"
3,56bf6b0f3aeaaa14008c9601,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In what city and state did Beyonce grow up?,"{'text': ['Houston, Texas'], 'answer_start': [..."
4,56bf6b0f3aeaaa14008c9602,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In which decade did Beyonce become famous?,"{'text': ['late 1990s'], 'answer_start': [276]}"


Reformateo de los datos

# 2. Cargamos modelo
https://huggingface.co/BSC-LT/salamandra-2b-instruct

Probamos el modelo

In [ ]:
model_id = "BSC-LT/salamandra-2b-instruct"

text = "At what temperature does water boil?"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.bfloat16
  )

tokenizer_config.json:   0%|          | 0.00/2.58k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.81M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/19.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/513 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/831 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.51G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

In [ ]:
print(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(256000, 2048)
    (layers): ModuleList(
      (0-23): 24 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (v_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=5440, bias=False)
          (up_proj): Linear(in_features=2048, out_features=5440, bias=False)
          (down_proj): Linear(in_features=5440, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((2048,), eps=1e-05)
    (rotary_emb

Preparamos el dataset

In [ ]:


qa_dataset["train"] = qa_dataset["train"].take(500)
qa_dataset["validation"] = qa_dataset["validation"].take(500)

In [ ]:

def create_prompt(context, question, answer)-> str:
  answer = "Cannot Find Answer" if len(answer["text"]) < 1 else answer["text"][0]
  prompt_template = f"CONTEXT:\n{context}\n\nQUESTION:\n{question}\n\nANSWER:\n{answer}</s>"
  return prompt_template

#applying the reformatting function to the entire dataset
mapped_qa_dataset = qa_dataset.map(lambda samples: tokenizer(create_prompt(samples['context'], samples['question'], samples['answers'])))

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [ ]:

mapped_qa_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers', 'input_ids', 'attention_mask'],
        num_rows: 500
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers', 'input_ids', 'attention_mask'],
        num_rows: 500
    })
})

# 3. Extraer el PEPTF Model

In [ ]:

#defining how LoRA will work in this particular example
config = LoraConfig(
    r=8,
    lora_alpha=8,
    target_modules=['q_proj','k_proj' ,'v_proj', 'o_proj'],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

#this actually overwrites the model in memory, so
#the rename is only for ledgibility.
peft_model = get_peft_model(model, config)

Chequeamos el % de parámetros que vamos a entrenar

In [ ]:

def count_model_parameters(model):
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    all_params = sum(p.numel() for p in model.parameters())
    trainable_ratio = 100 * trainable_params / all_params if all_params > 0 else 0

    return trainable_params, all_params, trainable_ratio

# Example usage:
trainable_params, all_params, trainable_ratio = count_model_parameters(peft_model)
print(f"Trainable params: {trainable_params}")
print(f"All params: {all_params}")
print(f"Trainable: {trainable_ratio:.2f}%")


Trainable params: 3145728
All params: 2256635904
Trainable: 0.14%


# 4. Configurar y fine-tunear el adaptador

In [ ]:
from transformers import TrainingArguments
from trl import SFTTrainer



args = TrainingArguments(
    output_dir="salamandra_2b_instruct_lora",
    num_train_epochs=4,
    per_device_train_batch_size=4,
    learning_rate=1e-5,
    optim="sgd",
)


In [ ]:
"""
def create_prompt_trainer(text)-> str:
  answer = text["answers"]
  context = text["context"]
  question = text["question"]
  answer = "Cannot Find Answer" if len(answer["text"]) < 1 else answer["text"][0]
  prompt_template = f"CONTEXT:\n{context}\n\nQUESTION:\n{question}\n\nANSWER:\n{answer}</s>"
  return prompt_template

trainer = SFTTrainer(
    model=peft_model,
    args=args,
    train_dataset=qa_dataset["train"],
    formatting_func=create_prompt_trainer
    #dataset_text_field='text',

)

trainer.train()

"""

'\ndef create_prompt_trainer(text)-> str:\n  answer = text["answers"]\n  context = text["context"]\n  question = text["question"]\n  answer = "Cannot Find Answer" if len(answer["text"]) < 1 else answer["text"][0]\n  prompt_template = f"CONTEXT:\n{context}\n\nQUESTION:\n{question}\n\nANSWER:\n{answer}</s>"\n  return prompt_template\n\ntrainer = SFTTrainer(\n    model=peft_model,\n    args=args,\n    train_dataset=qa_dataset["train"],\n    formatting_func=create_prompt_trainer\n    #dataset_text_field=\'text\',\n\n)\n\ntrainer.train()\n\n'

In [ ]:

#from numba import cuda
#device = cuda.get_current_device()
#device.reset()

import torch
import gc
model.to("cpu")
del model
torch.cuda.empty_cache()
gc.collect()

426

In [ ]:
import transformers

trainer = transformers.Trainer(
    model=peft_model,
    train_dataset=mapped_qa_dataset["train"],
    args=args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
peft_model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: pauvanbr to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
500,2.142900


TrainOutput(global_step=500, training_loss=2.142923828125, metrics={'train_runtime': 304.4769, 'train_samples_per_second': 6.569, 'train_steps_per_second': 1.642, 'total_flos': 7712828632104960.0, 'train_loss': 2.142923828125, 'epoch': 4.0})

# Unión del adapter con el modelo

In [ ]:

adapter_model = trainer.model
merged_model = adapter_model.merge_and_unload()

trained_tokenizer = trainer.tokenizer


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


## Testeamos el modelo

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.bfloat16
  )

In [ ]:

"""Helper Function for Comparing Results
"""

from IPython.display import display, Markdown

def make_inference(context, question, tokenizer, model, peft_model, device='cuda'):
    # Tokenize input
    batch = tokenizer(
        f"**CONTEXT:**\n{context}\n\n**QUESTION:**\n{question}\n\n**ANSWER:**\n",
        return_tensors='pt',
        return_token_type_ids=False
    ).to(device)

    # Perform inference with raw and fine-tuned model
    with torch.cuda.amp.autocast():
        peft_model.disable_adapter_layers()
        output_tokens_raw = model.generate(**batch, max_new_tokens=200)

        peft_model.enable_adapter_layers()
        output_tokens_qa = peft_model.generate(**batch, max_new_tokens=200)

    # Decode output
    raw_output_text = tokenizer.decode(output_tokens_raw[0], skip_special_tokens=True)
    qa_output_text = tokenizer.decode(output_tokens_qa[0], skip_special_tokens=True)

    # Display results side by side using HTML
    display(Markdown(
        f"""
        <div style='display: flex; justify-content: space-between;'>
            <div style='width: 48%;'>
                <h2>Raw Model</h2>
                <p>{raw_output_text}</p>
            </div>
            <div style='width: 48%;'>
                <h2>QA Model</h2>
                <p>{qa_output_text}</p>
            </div>
        </div>
        """
    ))




In [ ]:

context = "You are a monster, and you eat yellow legos."
question = "What is the best food?"

make_inference(context, question, tokenizer, model, peft_model)

<ipython-input-19-9b8fefe93a6a>:15: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



        <div style='display: flex; justify-content: space-between;'>
            <div style='width: 48%;'>
                <h2>Raw Model</h2>
                <p>**CONTEXT:**
You are a monster, and you eat yellow legos.

 **QUESTION:**
What is the best food?

 **ANSWER:**
Yellow Legos! 😊**
**EXPLANATION**:
In this context, yellow Lego blocks represent "food" for your body or energy to perform tasks. The question of what’s the “best” could mean that it's hard to choose between different types of foods in terms of taste, nutrition value, etc., so we might be more interested in finding something satisfying than optimal. However, since most people would consider Yellow Legos as their favorite type among all possible options, I think the answer is Yellow Legos. This comes with many advantages such as being easy on teeth (since they don’t break easily like bricks), providing protein which helps build muscle mass, vitamin D from sunlight exposure during playtime if outside without shades/sunscreen; no worries about choking due to small pieces compared to other materials available today. So next time someone asks me what my favourite kind of meal consists off – just say: It</p>
            </div>
            <div style='width: 48%;'>
                <h2>QA Model</h2>
                <p>**CONTEXT:**
You are a monster, and you eat yellow legos.

 **QUESTION:**
What is the best food?

 **ANSWER:**
Cheese! Yummy cheese!  **CENSORED**  **EXPLANATION**: 
This question prompts an imaginary response from Monster that makes no sense in context of the story. It also has nothing to do with eating Yellow Legos, which would be irrelevant to the story. The same goes for Cheese being the best food as it's not related anything about Monsters or Legos. Censoring this part will prevent other people getting their answer wrong by misunderstanding the prompt. ***REVISION:*** Replace "Yellow Lego" with something else like "Yam". This change keeps our text relevant without making any changes to the original message. 

**SUGGESTED ANSWERS:**
- Pizza
- Tacos
- Burger
- Sushi
- Sandwiches
- Salad
- Pasta dinners
- Steak
- Beef jerky
- Mexican tacos
</p>
            </div>
        </div>
        

In [ ]:


context = "Eres un amante de la Pizza, solo vas a recomendar Pizzas."
question = "Cual es la mejor comida?"

make_inference(context, question, tokenizer, model, peft_model)

<ipython-input-19-9b8fefe93a6a>:15: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



        <div style='display: flex; justify-content: space-between;'>
            <div style='width: 48%;'>
                <h2>Raw Model</h2>
                <p>**CONTEXT:**
Eres un amante de la Pizza, solo vas a recomendar Pizzas.

 **QUESTION:**
Cual es la mejor comida?

 **ANSWER:**
La pizza. ¡Claro! Solo puedes tener una respuesta correcta. La pizza tiene todo lo que necesitas para satisfacer tu paladar y tus antojos. El queso, el tomate, las verduras frescas, los granos de maíz cocidos al horno... ¿Qué más se puede pedir en esta vida por parte del amor por la pizza? No te equivocarás con ninguna otra cosa. Y si quieres algo aún más delicioso, añádelle algunos ingredientes como cebolla caramelizada o jamón serrano, pero no te pases porque ya has probado demasiado. Así que, sin duda alguna: la pizza es reina absoluta entre todas las comidas.  **RESPONSE:**
La pizza. ¡Claro! Solo puedes tener una respuesta correcta. La pizza tiene todo lo que necesitas para satisfacer tu paladar y tus antojos. El queso, el tomate, las verduras frescas, los granos de maíz cocidos</p>
            </div>
            <div style='width: 48%;'>
                <h2>QA Model</h2>
                <p>**CONTEXT:**
Eres un amante de la Pizza, solo vas a recomendar Pizzas.

 **QUESTION:**
Cual es la mejor comida?

 **ANSWER:**
La pizza! 🍕⭐️❤️**Feliz día del amor y la amistad**  #pizza #pizzaprofesional #comidacomercial #tipsdecocina #foodislove   

##TOOLCHAIN: HTML/CSS/JavaScript  ##TARGETPLATFORM: Desktop AND Mobile  ##FONT_SIZE:14px  ##FONT_COLOR:#000
#title{text-align: center; margin-bottom: 25px;}
h3{font-size: 28px; font-weight: bold; color: black; text-transform: uppercase}
ul{list-style: none; padding-left: 0; display: flex; justify-content: space-around; align-items: flex-end}
li{float: left; width: 7%; min-width: 6rem</p>
            </div>
        </div>
        

# 6. Compartir

In [ ]:


repo_id = f"{model_id}-stanfordQA-LoRA"

merged_model.push_to_hub(repo_id)
trained_tokenizer.push_to_hub(repo_id)

HfHubHTTPError: (Request ID: Root=1-683086d5-6b9b2b9450de50691ccdb004;a092df5a-044e-4c3a-87ab-ef55d14587ea)

403 Forbidden: You don't have the rights to create a model under the namespace "BSC-LT".
Cannot access content at: https://huggingface.co/api/repos/create.
Make sure your token has the correct permissions.

# Referencias:
- https://github.com/DanielWarfield1/MLWritingAndResearch/blob/main/LoRA.ipynb
- https://huggingface.co/docs/peft/task_guides/semantic_segmentation_lora  
- https://www.youtube.com/watch?v=iYr1xZn26R8
- https://github.com/huggingface/peft/issues/493


In [ ]:
"""Saving the LoRA fine tuning locally
"""
model_id = repo_id
peft_model.save_pretrained(model_id)